<a href="https://colab.research.google.com/github/m37335/kanagawa-exam/blob/master/part2_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
# 単語の分割の設定
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# マスクの設定
msk_model = BertForMaskedLM.from_pretrained("bert-base-cased")
# GPUを使用
msk_model.cuda()
msk_model.eval()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

### **２０１１年度問２（２）**

Many people climb this mountain (during) the summer every year.

In [ ]:
text = "[CLS] Many people climb this mountain during the summer every year. [SEP]"

In [ ]:
# 単語の分割
words = tokenizer.tokenize(text)
print(words)

['[CLS]', 'Many', 'people', 'climb', 'this', 'mountain', 'during', 'the', 'summer', 'every', 'year', '.', '[SEP]']


In [ ]:
msk_idx = 6
words[msk_idx] = "[MASK]"
print(words)

['[CLS]', 'Many', 'people', 'climb', 'this', 'mountain', '[MASK]', 'the', 'summer', 'every', 'year', '.', '[SEP]']


In [ ]:
# tokenをidに変換
word_ids = tokenizer.convert_tokens_to_ids(words)
word_tensor = torch.tensor([word_ids])
# print(word_tensor)

# マスクした部分の予測
x = word_tensor.cuda()
y = msk_model(x)
result = y[0]
# print(result.size())

_, msk_ids = torch.topk(result[0][msk_idx], k=10)
result_words = tokenizer.convert_ids_to_tokens(msk_ids.tolist())

# 予測した結果を出力
print(result_words)

['in', 'during', 'over', 'for', 'throughout', 'through', 'on', 'around', 'into', 'every']


### **2020年度問２（１）**

 Lions kill and eat large animals, so I'm (afraid) of lions. 

In [ ]:
text = "Lions kill and eat large animals, so I'm afraid of lions. "

In [ ]:
# 単語の分割
words = tokenizer.tokenize(text)
print(words)

msk_idx = 11
words[msk_idx] = "[MASK]"
print(words)

# tokenをidに変換
word_ids = tokenizer.convert_tokens_to_ids(words)
word_tensor = torch.tensor([word_ids])
# print(word_tensor)

# マスクした部分の予測
x = word_tensor.cuda()
y = msk_model(x)
result = y[0]
# print(result.size())

_, msk_ids = torch.topk(result[0][msk_idx], k=10)
result_words = tokenizer.convert_ids_to_tokens(msk_ids.tolist())

# 予測した結果を出力
print(result_words)

['Lions', 'kill', 'and', 'eat', 'large', 'animals', ',', 'so', 'I', "'", 'm', 'afraid', 'of', 'lions', '.']
['Lions', 'kill', 'and', 'eat', 'large', 'animals', ',', 'so', 'I', "'", 'm', '[MASK]', 'of', 'lions', '.']
['afraid', 'kind', 'thinking', 'not', 'part', 'like', 'out', 'sort', 'scared', 'proud']


### **2018年度問２（１）**

I'm learning about (traditional) Japanese events like Setsubun and Hinamatsuri for my speech next week.

In [ ]:
text = "I'm learning about traditional Japanese events like Setsubun and Hinamatsuri for my speech next week."

In [ ]:
# 単語の分割
words = tokenizer.tokenize(text)
#print(words)

msk_idx = 5
words[msk_idx] = "[MASK]"
#print(words)

# tokenをidに変換
word_ids = tokenizer.convert_tokens_to_ids(words)
word_tensor = torch.tensor([word_ids])
# print(word_tensor)

# マスクした部分の予測
x = word_tensor.cuda()
y = msk_model(x)
result = y[0]
# print(result.size())

_, msk_ids = torch.topk(result[0][msk_idx], k=10)
result_words = tokenizer.convert_ids_to_tokens(msk_ids.tolist())

# 予測した結果を出力
print(result_words)

['the', 'major', 'these', 'other', 'important', 'big', 'famous', 'great', 'historical', 'some']
